In [1]:
%matplotlib inline

/home/madeleine/anaconda2/envs/django/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/madeleine/anaconda2/envs/django/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#Import packages
import requests
import json
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import mlab
import seaborn as sns
import pandas as pd
from scipy.stats import poisson
from io import StringIO
sns.set_palette("husl")

In [160]:
prophages_dict = {}

with open("../data/active_accessions.txt", "r") as f:
    for line in f:
        acc = line.rstrip()
        print(acc)
        apiurl = "http://phaster.ca/phaster_api?acc={}".format(acc) 
        r=requests.get(apiurl)
        d=r.json()['summary']
        if d.find('Totally 0') != -1:
            prophages = False
            num_prophages = 0
        else:
            prophages = True
            # find number of prophages
            e='\n'.join(d.split('\n')[32:])
            data = pd.read_table(StringIO(e),skiprows=[1], delim_whitespace=True)
            num_prophages = sum(data['COMPLETENESS(score)'].str.contains('intact'))
        prophages_dict[acc] = num_prophages


NC_009012
NC_017304
NC_002655
NC_002695
NC_000913
NC_009801
NC_010468
NC_010473
NC_010498
NC_011415
NC_011741
NC_011748
NC_011745
NC_011742
NC_011993
NC_012759
NC_013008
NC_013364
NC_013941
NC_015172
NC_017906
NC_018661
NC_020163
NC_020518
NC_007946
NC_007779
NC_008563
NC_011993
NC_017634
NC_007946
NC_008563
NC_011742
NC_011745
NC_011993
NC_017634
NC_017632
NC_017628
NC_008563
NC_011993
NC_011993
NC_011993
NC_017632
NC_017634
NC_007946
NC_000913
NC_002655
NC_002695
NC_018650
NC_018661
NC_020163
NC_020518
NC_007779
NC_016902
NC_017625
NC_017628
NC_017632
NC_017634
NC_017635
NC_017638
NC_017641
NC_017660
NC_017660
NC_017906
NC_011742
NC_011745
NC_011745
NC_017628
NC_017628
NC_017632
NC_007946
NC_008563
NC_009012
NC_002655
NC_002695
NC_000913
NC_009801
NC_010468
NC_010473
NC_010498
NC_011353
NC_011415
NC_011751
NC_012759
NC_013008
NC_013361
NC_013941
NC_015172
NC_016902
NC_017625
NC_017626
NC_017633
NC_017641
NC_017656
NC_017660
NC_017906
NC_018658
NC_018650
NC_018661
NC_020163
NC_020518


In [162]:
prophages_dict

{'NC_000913': 4,
 'NC_002655': 12,
 'NC_002695': 13,
 'NC_007779': 2,
 'NC_007946': 6,
 'NC_008563': 9,
 'NC_009012': 5,
 'NC_009801': 3,
 'NC_010468': 3,
 'NC_010473': 3,
 'NC_010498': 2,
 'NC_011353': 14,
 'NC_011415': 7,
 'NC_011741': 3,
 'NC_011742': 7,
 'NC_011745': 8,
 'NC_011748': 6,
 'NC_011751': 5,
 'NC_011993': 4,
 'NC_012759': 2,
 'NC_013008': 14,
 'NC_013361': 14,
 'NC_013364': 13,
 'NC_013941': 11,
 'NC_015172': 0,
 'NC_016902': 7,
 'NC_017304': 0,
 'NC_017625': 1,
 'NC_017626': 5,
 'NC_017628': 12,
 'NC_017632': 5,
 'NC_017633': 8,
 'NC_017634': 3,
 'NC_017635': 7,
 'NC_017638': 3,
 'NC_017641': 9,
 'NC_017656': 8,
 'NC_017660': 6,
 'NC_017906': 10,
 'NC_018650': 6,
 'NC_018658': 7,
 'NC_018661': 8,
 'NC_020163': 5,
 'NC_020518': 0}

In [163]:
data

,REGION,REGION_LENGTH,COMPLETENESS(score),SPECIFIC_KEYWORD,REGION_POSITION,TRNA_NUM,TOTAL_PROTEIN_NUM,PHAGE_HIT_PROTEIN_NUM,HYPOTHETICAL_PROTEIN_NUM,PHAGE+HYPO_PROTEIN_PERCENTAGE,BACTERIAL_PROTEIN_NUM,ATT_SITE_SHOWUP,PHAGE_SPECIES_NUM,MOST_COMMON_PHAGE_NAME,MOST_COMMON_PHAGE_NUM,MOST_COMMON_PHAGE_PERCENTAGE,GC_PERCENTAGE
0,1,12Kb,incomplete(30),transposase,8238-20314,0,13,7,2,69.2%,4,no,5,PHAGE_Entero_Min27,2,15.3%,50.51%
1,2,21.3Kb,intact(100),"integrase,transposase,recombinase,lysin,lysis,...",563978-585280,0,26,23,0,88.4%,3,yes,12,PHAGE_Entero_lambda,7,26.9%,43.27%
2,3,18Kb,questionable(80),"integrase,terminase,tail,envelope",1200217-1218248,0,21,17,3,95.2%,1,yes,10,PHAGE_Entero_SfV,7,33.3%,44.65%
3,4,22.8Kb,questionable(70),"integrase,lysis,tail,transposase",1413683-1436526,0,27,18,5,85.1%,4,yes,13,PHAGE_Gifsy_2,4,14.8%,46.67%
4,5,33.2Kb,intact(100),"tail,terminase,envelope,lysin,transposase,inte...",1621271-1654566,0,39,23,11,87.1%,5,yes,15,PHAGE_Entero_mEp460,5,12.8%,45.62%
5,6,11.2Kb,incomplete(40),"integrase,flippase,tail",2471800-2483075,0,17,16,0,94.1%,1,yes,6,PHAGE_Entero_SfV,11,64.7%,44.39%
